### Import libraries

In [9]:
import pandas as pd
import numpy as np
import yfinance as yf

from datetime import datetime
import matplotlib.pyplot as plt

from fredapi import Fred

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score,accuracy_score
from sklearn.model_selection import TimeSeriesSplit

### Helpers

In [10]:
#########################
## Feature engineering ##
#########################

def create_ohlc_features(df, horizons=[2, 5, 10, 15, 20]):
    predictors = []

    for horizon in horizons:
        if len(df) < horizon:  # Skip if data size is smaller than the horizon
            continue

        # Rolling averages and ratios
        rolling_averages = df.rolling(horizon).mean()
        df[f"Close_Ratio_{horizon}"] = df["close"] / rolling_averages["close"]

        # Trend (cumulative sum of target over horizon)
        df[f"Trend_{horizon}"] = df["target"].shift(1).rolling(horizon).sum()

        # Volatility (rolling standard deviation)
        df[f"Volatility_{horizon}"] = df["close"].rolling(horizon).std()

        # Momentum (difference between current and lagged price)
        df[f"Momentum_{horizon}"] = df["close"] - df["close"].shift(horizon)

        # Exponential Moving Average (EMA)
        df[f"EMA_{horizon}"] = df["close"].ewm(span=horizon, adjust=False).mean()

        # Relative Strength Index (RSI)
        delta = df["close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(horizon).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(horizon).mean()
        df[f"RSI_{horizon}"] = 100 - (100 / (1 + gain / loss))

        # Rolling skewness and kurtosis (to capture distribution shape)
#         df[f"Skewness_{horizon}"] = df["close"].rolling(horizon).skew()
#         df[f"Kurtosis_{horizon}"] = df["close"].rolling(horizon).kurt()

        # Bollinger Bands (upper and lower bounds)
        rolling_std = df["close"].rolling(horizon).std()
        df[f"Bollinger_Upper_{horizon}"] = rolling_averages["close"] + 2 * rolling_std
        df[f"Bollinger_Lower_{horizon}"] = rolling_averages["close"] - 2 * rolling_std

        # Cumulative Return
        df[f"Cumulative_Return_{horizon}"] = df["close"].pct_change().rolling(horizon).sum()

        # Add features to the predictor list
        predictors += [
            f"Close_Ratio_{horizon}",
            f"Trend_{horizon}",
            f"Volatility_{horizon}",
            f"Momentum_{horizon}",
            f"EMA_{horizon}",
            f"RSI_{horizon}",
#             f"Skewness_{horizon}",
#             f"Kurtosis_{horizon}",
            f"Bollinger_Upper_{horizon}",
            f"Bollinger_Lower_{horizon}",
            f"Cumulative_Return_{horizon}",
        ]

    # Drop NA rows introduced by rolling and shifting, but only drop rows with all NaNs
    df = df.dropna(subset=predictors, how="all")

    return df, predictors



def create_calendar_features(df, date_col):
 
    # Ensure the date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])

    # Calendar features
    df["day_of_week"] = df[date_col].dt.dayofweek
    df["day_of_month"] = df[date_col].dt.day
    df["week_of_year"] = df[date_col].dt.isocalendar().week
    df["month"] = df[date_col].dt.month
    df["year"] = df[date_col].dt.year
    df["quarter"] = df[date_col].dt.quarter
    df["is_month_end"] = df[date_col].dt.is_month_end.astype(int)
    df["is_month_start"] = df[date_col].dt.is_month_start.astype(int)

    # Cyclical encoding
    df["day_of_week_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
    df["day_of_week_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    # List of generated features
    predictors = [
        "day_of_week",
        "day_of_month",
        "week_of_year",
        "month",
        "year",
        "quarter",
        "is_month_end",
        "is_month_start",
        "day_of_week_sin",
        "day_of_week_cos",
        "month_sin",
        "month_cos",
    ]

    return df, predictors




#################
## Backtesting ##
#################

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

def backtest(data, model, predictors, start=500, step=50):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)


#############
## helpers ##
#############

def convert_date_format(date_string):
    main_date_part = date_string.split(" (")[0]  
    parsed_date = datetime.strptime(main_date_part, "%b %d, %Y")
    return pd.to_datetime(parsed_date.strftime("%Y-%m-%d"))


### Bring data

In [11]:
fredapikey = "2128c9d039210886d2dbd0e7b35ac1c1"
fred = Fred(api_key=fredapikey)

start_date = '2023-06-01'
end_date = '2024-12-30'
# today_date = datetime.today().strftime('%Y-%m-%d')

df = yf.download('ZF=F', start=start_date,end=end_date)
df.reset_index(inplace=True)

df.drop(columns=['Adj Close'],inplace=True)
df.columns = ['date','open','high','low','close','volume']
df['date'] = pd.to_datetime(df['date'])
df.shape

[*********************100%***********************]  1 of 1 completed


(390, 6)

### Create Targets

In [12]:
df['tomorrow'] = df['close'].shift(-1)
df['target'] = (df['tomorrow'] > df['close']).astype(int)
df.head()

,date,open,high,low,close,volume,tomorrow,target
0,2023-06-01,108.460938,108.960938,108.195312,108.703125,27750,108.070312,0
1,2023-06-02,108.718750,108.718750,108.007812,108.070312,27467,108.132812,1
2,2023-06-05,107.976562,108.304688,107.734375,108.132812,6951,108.031250,0
3,2023-06-06,108.304688,108.304688,108.031250,108.031250,3653,107.765625,0
4,2023-06-07,108.132812,108.132812,107.625000,107.765625,1785,108.031250,1


### Baseline Model

In [ ]:
model = RandomForestClassifier(random_state=1)

train = df.iloc[:-300]
test = df.iloc[-300:]

predictors = ["close", "volume", "open", "high", "low"]
model.fit(train[predictors], train["target"])

In [ ]:
test['target'].hist()

In [ ]:
preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds) * 100

In [ ]:
predictions = pd.DataFrame({
    'target': test['target'],
    'prediction':preds
})

predictions.head(3)

In [ ]:
predictions.query('target ==1 and prediction ==1').shape[0]/ test[test['target'] == 1].shape[0]

### Backtesting

In [ ]:
predictions = backtest(df, model, predictors)
precision_score(predictions["target"], predictions["Predictions"])
predictions["target"].value_counts() / predictions.shape[0]


### Macro Data 

In [5]:
### Uneployment Rate ###

un_rate = pd.read_excel("Low frequency data.xlsx",sheet_name="Unemployment Rate")
un_rate['un_announcement'] = 1
un_rate.columns = ['date','time','actual','forecast','previous','un_announcement']
un_rate['date'] = un_rate['date'] .apply(convert_date_format)
un_rate['unempl_change'] = un_rate['forecast'] - un_rate['previous']
un_rate["next_announcement_date"] = un_rate["date"].shift(-1)
un_rate["days_until_next_announcement"] = (un_rate["next_announcement_date"] - un_rate["date"]).dt.days
un_rate.drop(columns=['time','previous','forecast','actual','next_announcement_date'],inplace=True)

### CPI ###
cpi = pd.read_excel("Low frequency data.xlsx",sheet_name="CPI")
cpi['cpi_announcement'] = 1
cpi.columns = ['date','time','actual','forecast','previous','cpi_announcement']
cpi['date'] = cpi['date'].apply(convert_date_format)
cpi['cpi_change'] = cpi['forecast'] - cpi['previous']
cpi["next_announcement_date"] = cpi["date"].shift(-1)
cpi["days_until_next_announcement"] = (cpi["next_announcement_date"] - cpi["date"]).dt.days
cpi.drop(columns=['time','previous','forecast','actual','next_announcement_date'],inplace=True)

### Fed rates ### 
interest_rates = pd.read_excel('Low frequency data.xlsx', sheet_name='Fed Interest Rate')
interest_rates['interest_announcement'] = 1
interest_rates.columns = ['date','time','actual','forecast','previous','interest_announcement']
interest_rates['interest_change'] = interest_rates['forecast'] - interest_rates['previous']
interest_rates["next_announcement_date"] = interest_rates["date"].shift(1)
interest_rates["days_until_next_announcement"] = (interest_rates["next_announcement_date"] - interest_rates["date"]).dt.days
interest_rates.drop(columns=['time','previous','forecast','actual'],inplace=True)


In [6]:
cpi

,date,cpi_announcement,cpi_change,days_until_next_announcement
0,2024-11-13,1,NaN,-34.0
1,2024-10-10,1,-0.001,-29.0
2,2024-09-11,1,0.000,-28.0
3,2024-08-14,1,0.003,-34.0
4,2024-07-11,1,0.001,-29.0
5,2024-06-12,1,-0.002,-28.0
6,2024-05-15,1,0.000,-35.0
7,2024-04-10,1,-0.001,-29.0
8,2024-03-12,1,0.001,-28.0
9,2024-02-13,1,0.000,-33.0


In [ ]:
interest_rates

### Feature Addition

In [13]:
df = df.merge(cpi, on='date', how='left')
# df = df.merge(interest_rates, on='date', how='left')
# df = df.merge(un_rate, on='date', how='left')
# df.fillna(0,inplace=True)
# df.shape

In [ ]:
df, calendar_predictors = create_calendar_features(df, date_col='date')
df.head()

In [ ]:
# df, ohlc_predictors = create_ohlc_features(df)
df, ohlc_predictors = create_ohlc_features(df)
df.shape

In [ ]:
list(df.columns[8:14])

In [ ]:
fed_predictors = list(df.columns[8:14])

In [ ]:
predictors = calendar_predictors + fed_predictors + ohlc_predictors 


In [ ]:
df.isna().sum().sort_values()

In [ ]:
df.dropna(inplace=True)

In [ ]:
model = RandomForestClassifier(random_state=1)
predictions = backtest(df, model, predictors)
precision_score(predictions["target"], predictions["Predictions"])


In [ ]:
predictions

In [ ]:
predictions["target"].value_counts() / predictions.shape[0]


In [ ]:
feature_importances.head(30)

In [ ]:
# Get feature importances
importances = model.feature_importances_
feature_importances = pd.DataFrame({
    'Feature': predictors,
    'Importance': importances
}).sort_values(by="Importance", ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importances['Feature'], feature_importances['Importance'], color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances from Random Forest')
plt.gca().invert_yaxis()  
plt.show()

### Feature Selection

### Evaluation